<a href="https://colab.research.google.com/github/ljkennett-20/fullcontrol/blob/master/models/colab/design_template_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FullControl design template

*<<< check out demo models [here](https://github.com/FullControlXYZ/fullcontrol/tree/master/models/README.md) >>>*
  
press ctrl+F9 to run all cells in this notebook, or press shift+enter to run each cell sequentially

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*
### be patient :)

the next code cell may take a while because running it causes several things to happen:
- connect to a google colab server -> download the fullcontrol code -> install the fullcontrol code

check out [other tutorials](https://github.com/FullControlXYZ/fullcontrol/blob/master/tutorials/README.md) to understand the python code for the FullControl design

In [1]:
if 'google.colab' in str(get_ipython()):
  !pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
from google.colab import files

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# printer/gcode parameters

design_name = 'my_design'
nozzle_temp = 210
bed_temp = 40
print_speed = 1000
fan_percent = 100
printer_name='prusa_i3' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [3]:
# design parameters

EW = 0.8 # extrusion width
EH = 0.3 # extrusion height (and layer height)
initial_z = EH*0.6 # initial nozzle position is set to 0.6x the extrusion height to get a bit of 'squish' for good bed adhesion
layers = 50

In [15]:
from math import sin, pi
steps = []
centre_point = fc.Point(x=10, y=10, z=0)
a = 10
b = 5
start_angle = 0
segments = 32
clockwise = True
amp = 0.5

for i in range(100):
  if i < 50:
    centre = fc.Point(x=10+amp*sin(i*pi/12), y=10+amp*sin(i*pi/12), z=i*EH)
    layer_steps = fc.ellipseXY(centre, a, b, start_angle, segments, clockwise)
  else:
    centre = fc.Point(x=10, y=10, z=i*EH)
    new_a = a+ amp*2*sin(i*pi/6)
    new_b = b+ amp*2*sin(i*pi/6)
    layer_steps = fc.ellipseXY(centre, new_a, new_b, start_angle, segments, clockwise)
  steps.extend(layer_steps)
# steps.append(fc.PlotAnnotation(point=steps[-1], label="start/end"))
# steps.append(fc.PlotAnnotation(point=steps[1], label="first point after start"))
# steps.append(fc.PlotAnnotation(point=centre_point, label="centre"))
fc.transform(steps, 'plot', fc.PlotControls(color_type='print_sequence', style='line'))


In [4]:
# generate the design (make sure you've run the above cells before running this cell)

steps = []
for layer in range(layers):
  steps.append(fc.Point(x=50, y=50, z=initial_z+layer*EH))
  steps.append(fc.Point(x=100, y=50, z=initial_z+layer*EH))
  steps.append(fc.Point(x=100, y=100, z=initial_z+layer*EH))
  steps.append(fc.Point(x=50, y=100, z=initial_z+layer*EH))
  steps.append(fc.Point(x=50, y=50, z=initial_z+layer*EH))

# instead of the above for-loop code, you can create the exact same design using built-in FullControl functions (uncomment the next two lines):
# rectangle_steps = fc.rectangleXY(fc.Point(x=50, y=50, z=initial_z), 50, 50)
# steps = fc.move(rectangle_steps, fc.Vector(z=EH), copy=True, copy_quantity=layers)

In [5]:
# preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
# fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))


In [ ]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,

    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)
open(f'{design_name}.gcode', 'w').write(gcode)
files.download(f'{design_name}.gcode')

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109.
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol